In [1]:
using DiffEqJump, StochasticDiffEq, DiffEqCallbacks
using StatsBase, Parameters, Plots, Distributions
using BenchmarkTools

### Callable Struct

In [4]:
# This stuff needs to be at top level
struct AffectIndex{F1, F2}
  affect_index!::F1
  index::F2
end
function (p::AffectIndex)(integrator)
    return p.affect_index!(integrator, p.index)
end

function enclose_this()
    function μ_SDE(du,u,p,t)
      du .= p.μ
    end
    function σ_SDE(du,u,p,t)
      du .= p.σ
    end
    
    p = (μ = 0.01, σ = 0.1, N = 300) # if all constant
    T = 10.0  # maximum time length
    x_iv = rand(p.N)  # just draws from the inital condition

    prob = SDEProblem(μ_SDE, σ_SDE, x_iv ,(0.0, T), p)
    rate(u,p,t) = 0.2
    affect_index!(integrator, index) = (integrator.u[index] = max(integrator.u[index], integrator.u[rand(1:integrator.p.N)]) )
    jumps = [ConstantRateJump(rate,AffectIndex(affect_index!, i)) for i in 1:p.N]
    jump_prob = JumpProblem(prob,DirectFW(),JumpSet((), jumps, nothing, nothing))
    sol = solve(jump_prob, SRIW1());
end

enclose_this (generic function with 1 method)

In [8]:
@benchmark enclose_this()

BenchmarkTools.Trial: 
  memory estimate:  21.39 MiB
  allocs estimate:  378341
  --------------
  minimum time:     37.033 ms (7.54% GC)
  median time:      43.452 ms (8.80% GC)
  mean time:        44.567 ms (11.20% GC)
  maximum time:     129.983 ms (66.50% GC)
  --------------
  samples:          113
  evals/sample:     1

### Closure

In [14]:
function closure_approach()
    function μ_SDE(du,u,p,t)
      du .= p.μ
    end
    function σ_SDE(du,u,p,t)
      du .= p.σ
    end
    
    p = (μ = 0.01, σ = 0.1, N = 300) # if all constant
    T = 10.0  # maximum time length
    x_iv = rand(p.N)  # just draws from the inital condition

    prob = SDEProblem(μ_SDE, σ_SDE, x_iv ,(0.0, T), p)
    rate(u,p,t) = 0.2
    affect_index!(integrator, index) = (integrator.u[index] = max(integrator.u[index], integrator.u[rand(1:integrator.p.N)]) )
    jumps = [ConstantRateJump(rate, integrator -> affect_index!(integrator, i)) for i in 1:p.N]
    jump_prob = JumpProblem(prob,DirectFW(),JumpSet((), jumps, nothing, nothing))
    sol = solve(jump_prob, SRIW1());
end

closure_approach (generic function with 1 method)

In [15]:
@benchmark closure_approach()

BenchmarkTools.Trial: 
  memory estimate:  22.25 MiB
  allocs estimate:  386829
  --------------
  minimum time:     38.934 ms (8.73% GC)
  median time:      43.642 ms (8.69% GC)
  mean time:        44.168 ms (9.75% GC)
  maximum time:     65.384 ms (18.08% GC)
  --------------
  samples:          114
  evals/sample:     1